<a href="https://colab.research.google.com/github/Youngdeuk-Koo/KNLP/blob/main/%ED%8A%B8%EB%9E%9C%EC%8A%A4%ED%8F%AC%EB%A8%B8%EB%A5%BC_%EC%9D%B4%EC%9A%A9%ED%95%9C_%EC%B1%97%EB%B4%87_%EB%A7%8C%EB%93%A4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab
!bash install_mecab-ko_on_colab190912.sh
%cd ../

!pip install gensim
!pip install gdown
!pip install sentencepiece

#0.프로젝트 진행과정과 생겼던 문제점 및 해결 방법

###1.전처리 관련(큰 문제 없었음)
    1.1 데이터 자체가 워낙 깔끔하여 전처리 할 부분이 거의 없었음
    1.2 중복 제거와 기타 특수문자 제거만 실행
 
###2.Word2Vector를 이용한 문장 Augmaentation으로 데이터 부풀리기

    2.1 유사 단어 변경시 자음이 따로 분리 되는 현상이 있었음 (ex:본다 -> 보ㄴ다)

    2.2 오류로 나온 자음 전체를 제거 하자 문법상 큰 문제가 생겨서 취소

    2.3 단어장에 없는 단어를 변경 시 파이썬 에러

    2.4 expect를 이용하여 None으로 변경처리

    2.5 공백 기준으로 유사 단어 변경시 너무 많은 None으로 교체 되는 현상 확인

    2.6 기준이 되는 문장을 메캅으로 형태소 분리 후 유사 단어 변경 적용

    2.7 None값이 현전히 줄어들며 개선

    2.8 생성된 문자를 다시 전처리 함수로 넣는 과정에서 더이상 lower() 함수에서 소문자가 없다며 에러

    2.9 lower() 함수도 expect pass 처리 

    2.10 변경된 문장을 기존 문장 밑으로 append 

    2.11 한번으론 부족하여 2중 포문을 이용하여 3~6번의 유사 문장을 생성후 기존 문장에 append

    2.12 쌍을 유지한채로 None값을 지우기 위해 여러 연산자와 방법을 고려

    2.13 None값이 제거된 각 쌍을 mecab으로 형태소 분리까지 했지만 sentencepiece 단어 사전 학습에서 보캡 사이즈가 너무 적게 나오는 현상이 있어 취소

    2.14 7700개의 문장을 약 42000개로 부풀리기 성공

###3.sentence piece를 이용한 단어 사전 생성및 토크나이저 변경 
    
    3.1 mecab을 이용한 형태소 분리후 학습시 , '가 같이 학습되는 현상 확인

    3.2 mecab을 취소 후 단어 사전 학습

###4.트랜스포머의 하이퍼 파라미터와 학습율 설정

    4.1 주어진 파라미터의 연산이 맞지 않는 경우 에러 발생(멀티헤드어텐션)

    4.2 하이퍼 파라미터가 멀티 헤드 어텐션 값(8)의 배수인 경우가 가장 적절

    4.3 학습시 Loss값이 Nan으로 나오는 현상 확인

    4.4 적용된 하이퍼 파라미터에 비해서 학습율이 너무 높아 생기는 현상으로 추정(확실치 않음)

    4.5 warp step 2000으로 감소후 해결

    4.6 Loss율이 제대로 떨어지지 않고 학습이 제대로 되지 않는 현상 확인

    4.7 데이터의 문제로 생각해 많은 것을 변경해 보았으나 별 의미가 없었음

    4.8 Dropout 값이 너무 높아(0.4) 생기는 문제로 파악

    4.9 Dropout 값을 0.2 ~ 0.3으로 조정 후 해결 

###5.적당한 에포크 찾기

    5.1 드랍아웃으로 인해 과적합 우려는 적었지만 학습속도가 현저히 느렸음

    5.2 10부터 50까지 다양한 에포크를 시행

    5.3 최소 60정도의 에포크만큼 학습해야 낮은 로스율이 가능할것이라 생각 됨

###6.챗봇 성공

    6.1 학습된 단어사전이 한개 뿐이라 적용이 편했다

    6.2 문장 출력 성공




In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import tensorflow_datasets as tfds
import sentencepiece as spm
import tensorflow as tf
import gensim
import gdown
import zipfile
from konlpy.tag import Mecab
mecab = Mecab()

#1.챗봇 데이터 및 ko.bin 데이터 크롤링

In [ ]:
google_path = 'https://drive.google.com/uc?id='
file_id = '0B0ZXk88koS2KbDhXdWg1Q2RydlU'
output_name = 'ko.zip'
gdown.download(google_path + file_id,output_name, quiet=False)

path_to_zip_file = '/content/ko.zip'

with zipfile.ZipFile(path_to_zip_file, 'r') as zip_ref:
    zip_ref.extractall()

Downloading...
From: https://drive.google.com/uc?id=0B0ZXk88koS2KbDhXdWg1Q2RydlU
To: /content/ko.zip
100%|██████████| 80.6M/80.6M [00:00<00:00, 129MB/s] 


In [ ]:
path_url = "https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv"

urllib.request.urlretrieve(path_url, filename="ChatBotData.csv")
df = pd.read_csv('ChatBotData.csv')
df

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


#2.데이터 중복, Null 제거 및 불필요한 데이터 정리 

In [ ]:
df.drop_duplicates(subset=['Q'], inplace=True)
df.drop_duplicates(subset=['A'], inplace=True)

df.reset_index(inplace=True, drop=True)
del df['label']

print('데이터 갯수 :', len(df))
print(df.isnull().sum())

데이터 갯수 : 7731
Q    0
A    0
dtype: int64


#3.Word2Vec ko.bin 언어 사전 적용 및 확인

In [ ]:
model = gensim.models.Word2Vec.load('/content/ko.bin')
print(model)
print(model.wv.most_similar("강아지", topn=2))

Word2Vec(vocab=30185, size=200, alpha=0.025)
[('고양이', 0.7290452718734741), ('거위', 0.7185635566711426)]


#4.전처리 함수

In [ ]:
def build_corpus(src):

    try:    # 소문자가 있을시
        src = src.lower()

    except: # 소문자가 없을시
        pass
    
    src = re.sub('[-=+,#/\:;^$@*\’"“”※~&%ㆍ_』\\‘|\(\)\[\]\<\>`\'…》.]','', str(src))
    src = re.sub(r"([?!])", r" \1 ", str(src))
   
    return src

#5.Data Augmentation 생성 및 None값 제거 함수

In [ ]:
def lexical_sub(sentence, word2vec):
    import random

    res = ""

    try:
        _from = random.choice(sentence)
        _to = model.wv.most_similar(_from)[0][0]

    except: #단어장에 없는 단어
        return None
    
    for tok in sentence:
        if tok is _from: 
            res += _to + " "
        else: 
            res += tok + " "
    
    return res

In [ ]:
from tqdm import tqdm_notebook

def sentence_generator(src, tgt, num=3):    

    for i in range(num):

        for idx in tqdm_notebook(range(len(df))):

            init_src = lexical_sub(mecab.morphs(src[idx]), model.wv)
            init_tgt = lexical_sub(mecab.morphs(tgt[idx]), model.wv)

            src.append(build_corpus(init_src))
            tgt.append(build_corpus(init_tgt))

    que = [w for w, s in zip(src, tgt) if 'None' not in w and 'None' not in s]
    ans = [s for w, s in zip(src, tgt) if 'None' not in w and 'None' not in s]

    print('\n', 'Data Augmentation Complete!', '\n')
    print('>>> Questions Data Size :', len(que))
    print('>>> Answers Data Size :', len(ans), '\n')

    return [w for w in zip(que, ans)]

pair = sentence_generator([build_corpus(w) for w in df['Q']], [build_corpus(w) for w in df['A']], 6)
print('>>> Sample Questions Data :', pair[::7731])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/7731 [00:00<?, ?it/s]

  0%|          | 0/7731 [00:00<?, ?it/s]

  0%|          | 0/7731 [00:00<?, ?it/s]

  0%|          | 0/7731 [00:00<?, ?it/s]

  0%|          | 0/7731 [00:00<?, ?it/s]

  0%|          | 0/7731 [00:00<?, ?it/s]


 Data Augmentation Complete! 

>>> Questions Data Size : 41731
>>> Answers Data Size : 41731
Sample Questions Data : [('12시 땡 ! ', '하루가 또 가네요'), ('3 박 4 일 놀 ㄹ래 가 고 싶 다 ', '여행 은데 언제나 좋 죠 '), ('워낙 잡 고 싶 지만 도저히 잡 지 못해 ', '각기 를 위한 선택 이 길 바랄 게요 '), ('남친 이 나 보다 눈물 이 던 많 아 ', '감수 성 이 풍부 한두 사람 이 군요 '), ('외교 활동 하 다 만났 어 ', '꿩 씹 고 알 먹 고 네요 '), ('간질 이 주저 앉 은 듯 합니다 ', '충격 이 컸 을 거 라고 생각 해요 ')]


#6.Sentence piece 언어 사전 생성 함수

In [ ]:
def generate_tokenizer(corpus,
                       vocab_size,
                       name ="chatbot",
                       pad_id = 0, # pad token의 일련번호
                       bos_id = 1, # 문장의 시작을 의미하는 bos token (<s>)의 일련번호
                       eos_id = 2, # 문장의 끝을 의미하는 eos token (</s>)의 일련번호
                       unk_id = 3): # unk token의 일련번호
    
    file = "./%s_corpus.txt" % name
    model = "%s_spm" % name

    # with open(file, 'w', encoding='utf-8') as f:
    #     f.write('\n'.join(map(str, corpus)))

    with open(file, 'w', encoding='utf-8') as f:
        for row in corpus: f.write(str(row) + '\n')
    

    spm.SentencePieceTrainer.Train(
        f'--input=./{file} --model_prefix={model} --vocab_size={vocab_size}' +
        f'--pad_id=={pad_id} --bos_id={bos_id} --eos_id={eos_id} --unk_id={unk_id}')
        
        

    tokenizer = spm.SentencePieceProcessor()
    tokenizer.Load('%s.model' % model)
    return tokenizer

In [ ]:
VOCAB_SIZE = 11942
tokenizer = generate_tokenizer(pair, VOCAB_SIZE)
tokenizer.set_encode_extra_options("bos:eos")    # 문장 양 끝에 <s>, </s>추가

True

#7.문장 인코딩

In [ ]:
src = []
tgt = []

for Q, A in tqdm_notebook(pair):

    src_tokens = tokenizer.encode_as_ids(str(Q))
    tgt_tokens = tokenizer.encode_as_ids(str(A))

    src.append(src_tokens)
    tgt.append(tgt_tokens)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  after removing the cwd from sys.path.


  0%|          | 0/41731 [00:00<?, ?it/s]

8.패딩 및 확인

In [ ]:
enc_train = tf.keras.preprocessing.sequence.pad_sequences(src, padding='post', maxlen=max([len(w) for w in src]))
dec_train = tf.keras.preprocessing.sequence.pad_sequences(tgt, padding='post', maxlen=max([len(w) for w in tgt]))

print('Questions Padding Size :', max([len(w) for w in enc_train]))
print('Answers Padding Size :', max([len(w) for w in dec_train]),'\n')
print('Questions Vector size :', len(enc_train))
print('Answers Vector size :', len(dec_train),'\n')
print('Sample Padding Data :', enc_train[20000])

Questions Padding Size : 40
Answers Padding Size : 43 

Questions Vector size : 41731
Answers Vector size : 41731 

Sample Padding Data : [   1  191 1338   13 5861    2    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0]


#8.포지셔널 인코딩 함수

In [ ]:
# Positional Encoding 구현
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, int(i) / d_model)

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])

    return sinusoid_table

#9.문장을 가릴 마스크 생성 함수

In [ ]:
# Mask  생성하기
def generate_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]

def generate_causality_mask(src_len, tgt_len):
    mask = 1 - np.cumsum(np.eye(src_len, tgt_len), 0)
    return tf.cast(mask, tf.float32)

def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_mask = generate_padding_mask(tgt)

    dec_causality_mask = generate_causality_mask(tgt.shape[1], tgt.shape[1])
    dec_mask = tf.maximum(dec_mask, dec_causality_mask)

    dec_enc_causality_mask = generate_causality_mask(tgt.shape[1], src.shape[1])
    dec_enc_mask = tf.maximum(enc_mask, dec_enc_causality_mask)

    return enc_mask, dec_enc_mask, dec_mask

#10.멀티 헤드 어텐션 함수

In [ ]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        self.depth = d_model // self.num_heads

        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)

        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)
        out = tf.matmul(attentions, V)

        return out, attentions


    def split_heads(self, x):
        bsz = x.shape[0]
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])

        return split_x

    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x


    def call(self, Q, K, V, mask):
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)

        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)

        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)

        out = self.combine_heads(out)
        out = self.linear(out)

        return out, attention_weights

#11.피드포워드 신경망 함수

In [ ]:
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)

        return out

#12.인코딩 레이어 함수

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):

        """
        Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, enc_attn

#13.디코딩 레이어 함수

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, enc_out, causality_mask, padding_mask):

        """
        Masked Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        """
        Multi-Head Attention
        """
        residual = out
        out = self.norm_2(out)
        out, dec_enc_attn = self.dec_self_attn(out, enc_out, enc_out, causality_mask)
        out = self.do(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn

#14.인코더 함수

In [ ]:
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):
        out = x

        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)

        return out, enc_attns

#15.디코더 함수

In [ ]:
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]


    def call(self, x, enc_out, causality_mask, padding_mask):
        out = x

        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, causality_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

#16.최종 트랜스포머 함수

In [ ]:
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()

        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        out = self.do(out)

        return out


    def call(self, enc_in, dec_in, enc_mask, causality_mask, dec_mask):
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)

        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, causality_mask, dec_mask)

        logits = self.fc(dec_out)

        return logits, enc_attns, dec_attns, dec_enc_attns

#17.트랜스포머 모델 파라미터값 설정(논문 환경 그대로 설정)

In [ ]:
# 주어진 하이퍼파라미터로 Transformer 인스턴스 생성
transformer = Transformer(
    n_layers=6,
    d_model=512,
    n_heads=8,
    d_ff=2048,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=0.3,
    shared_fc=True,
    shared_emb=True)

#18.학습율 스케쥴러 생성 함수

In [ ]:
# Learning Rate Scheduler 구현
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()
        
        self.d_model = d_model
        self.warmup_steps = warmup_steps
    
    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)

        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

#19.학습율 및 옵티마이저(아담) 설정

In [ ]:
# Learning Rate 인스턴스 선언 & Optimizer 구현
d_model=512

learning_rate = LearningRateScheduler(d_model, warmup_steps=4000)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)

#20.로스 함수

In [ ]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

#21.훈련 스텝 함수(마스크값 설정)

In [ ]:
# Train Step정의
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1] # Decoder의 inpurt
    gold = tgt[:, 1:] # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)
    
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

#22.훈련 시작(에포크 50)

In [ ]:
from tqdm import tqdm_notebook
import random

batch_size = 64
epochs = 50

for epoch in range(epochs):
    total_loss = 0

    idx_list = list(range(0, enc_train.shape[0], batch_size))
    random.shuffle(idx_list)
    t = tqdm_notebook(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
        train_step(enc_train[idx:idx+batch_size],
                   dec_train[idx:idx+batch_size],
                   transformer,
                   optimizer)
        
        total_loss += batch_loss

        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


  0%|          | 0/653 [00:00<?, ?it/s]

  0%|          | 0/653 [00:00<?, ?it/s]

  0%|          | 0/653 [00:00<?, ?it/s]

  0%|          | 0/653 [00:00<?, ?it/s]

...

  0%|          | 0/653 [00:00<?, ?it/s]

  0%|          | 0/653 [00:00<?, ?it/s]

  0%|          | 0/653 [00:00<?, ?it/s]

  0%|          | 0/653 [00:00<?, ?it/s]

  0%|          | 0/653 [00:00<?, ?it/s]

#23.새로운 입력문장 인코딩 및 평가 함수

In [ ]:
def evaluate(sentence, model):
    sentence = build_corpus(sentence)
    pieces = tokenizer.encode_as_pieces(str(sentence))
    tokens = tokenizer.encode_as_ids(str(sentence))

    _input = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')
    
    ids = []
    output = tf.expand_dims([tokenizer.bos_id()], 0)
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(_input, output)

        predictions, enc_attns, dec_attns, dec_enc_attns = model(_input, 
                                                                 output,
                                                                 enc_padding_mask,
                                                                 combined_mask,
                                                                 dec_padding_mask)

        predicted_id = tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()

        if tokenizer.eos_id() == predicted_id:
            result = tokenizer.decode_ids(ids)
            return pieces, result, enc_attns, dec_attns, dec_enc_attns

        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = tokenizer.decode_ids(ids)

    return pieces, result, enc_attns, dec_attns, dec_enc_attns

#24.번역 함수

In [ ]:
def translate(sentence, model):
    pieces, result, enc_attns, dec_attns, dec_enc_attns = evaluate(sentence, model)

    print('Input : %s' % (sentence))
    print('Output : {}'.format(result),'\n')

#25.번역 확인(성공)

In [ ]:
sentence = (
'지루하다 놀러가고 싶어',
'배가 너무 고파요',
'재밌는 영화 뭐 없을까?',
'날씨가 너무 추워요')

In [ ]:
for i in sentence:
    translate(str(i), transformer)

Input : 지루하다 놀러가고 싶어
Output : 여행을 떠나보세요 

Input : 배가 너무 고파요
Output : 뭐 좀 챙겨드세요 

Input : 재밌는 영화 뭐 없을까?
Output : 다큐멘터리 관전해보세요 

Input : 날씨가 너무 추워요
Output : 집밖에 나가기가 힘들것 같아요 

